In [1]:
# Import dependencies
import pandas as pd
import numpy as np

import tensorflow
tensorflow.keras.__version__

'2.3.0-tf'

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

# Import Dataset

In [3]:
nineFive_random_df = pd.read_csv('dataset/95k-random.csv')
nineFive_random_df.head(5)

,id,account_age,no_follower,no_following,no_userfavourites,no_lists,no_tweets,no_retweets,no_hashtag,no_usermention,no_urls,no_char,no_digits,class
0,1,1235,12,31,0,0,108,0,1,0,1,30,0,spammer
1,2,695,126,569,16,0,5048,1,1,2,1,109,1,spammer
2,3,448,4,63,0,1,860,0,0,0,1,36,4,spammer
3,4,1322,8,294,2,0,237,0,0,0,1,35,2,spammer
4,5,111,1461,1365,438,10,6894,0,1,0,1,55,6,spammer


In [4]:
nineFive_random_df = nineFive_random_df.drop('id', axis=1)

# Data Pre-Processing

In [5]:
nineFive_random_df['class'].value_counts()

non-spammer    95000
spammer         5000
Name: class, dtype: int64

In [6]:
X = nineFive_random_df.drop('class', axis=1)
y = nineFive_random_df['class']
print(X.shape, y.shape)

(100000, 12) (100000,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [8]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
# Create model, add layers & hidden nodes
model = Sequential()
model.add(Dense(units=36, activation='relu', input_dim=12))
model.add(Dense(units=36, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [25]:
# Compile and fit the model
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 36)                468       
_________________________________________________________________
dense_4 (Dense)              (None, 36)                1332      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 74        
Total params: 1,874
Trainable params: 1,874
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
2344/2344 - 9s - loss: 0.1912 - accuracy: 0.9499
Epoch 2/60
2344/2344 - 8s - loss: 0.1723 - accuracy: 0.9509
Epoch 3/60
2344/2344 - 8s - loss: 0.1666 - accuracy: 0.9512
Epoch 4/60
2344/2344 - 8s - loss: 0.1636 - accuracy: 0.9518
Epoch 5/60
2344/2344 - 8s - loss: 0.1609 - accuracy: 0.9520
Epoch 6/60
2344/2344 - 10s - loss: 0.1581 - accuracy: 0.9527
Epoch 7/60
2344/2344 - 9s - loss: 0.1554 - accuracy: 0.9535
Epoch 8/60
2344/2344 - 9s - loss: 0.1534 - accuracy: 0.9540
Epoch 9/60
2344/2344 - 8s - loss: 0.1517 - accuracy: 0.9548
Epoch 10/60
2344/2344 - 10s - loss: 0.1504 - accuracy: 0.9552
Epoch 11/60
2344/2344 - 8s - loss: 0.1481 - accuracy: 0.9557
Epoch 12/60
2344/2344 - 9s - loss: 0.1480 - accuracy: 0.9558
Epoch 13/60
2344/2344 - 9s - loss: 0.1465 - accuracy: 0.9563
Epoch 14/60
2344/2344 - 8s - loss: 0.1454 - accuracy: 0.9567
Epoch 15/60
2344/2344 - 9s - loss: 0.1446 - accuracy: 0.9569
Epoch 16/60
2344/2344 - 7s - loss: 0.1439 - accuracy: 0.9571
Epoch 17/60
2344/2344 - 8s - lo

In [28]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Loss: {model_loss}, Accuracy: {model_accuracy}")

782/782 - 3s - loss: 0.1344 - accuracy: 0.9602
Loss: 0.13440033793449402, Accuracy: 0.9602400064468384


In [21]:
encoded_predictions = model.predict_classes(X_test_scaled[:100])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [22]:
predict = model.predict(X_test_scaled[:100])
labels = np.argmax(predict, axis=1)
prediction_labels = label_encoder.inverse_transform(labels)

In [23]:
print(f"Predicted Classes: {prediction_labels}")
print(f"Actual labels: {list(y_test[:100])}")

Predicted Classes: ['non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-